In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import json
import open3d as o3d

from dust3r.inference import inference_with_mask, create_gaussian_kernel
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.cloud_opt.base_opt import global_alignment_loop
from masked_dust3r.scripts.utils.math import *
from masked_dust3r.scripts.utils.image import *


DATA_PATH = "/dust3r/masked_dust3r/data/jackal_irl_one_spin"
IMG_FILE_EXTENSION = ".png"
MASK_FILE_EXTENSION = ".png"

INIT_FRAMES = 50
NEW_FRAMES = 10
PREVIOUS_FRAMES = 40
TOTAL_FRAMES = 300

INIT_WEIGHT_FOCAL = 0.0001 * 0
INIT_WEIGHT_Z = 0.01
INIT_WEIGHT_ROT = 0.01 
INIT_WEIGHT_TRANS_SMOOTHNESS = 0.001 * 0
INIT_WEIGHT_ROT_SMOOTHNESS = 0.001 * 0

NEW_WEIGHT_FOCAL = 0.1 * 0
NEW_WEIGHT_Z = 0.1
NEW_WEIGHT_ROT = 0.1
NEW_WEIGHT_TRANS_SMOOTHNESS = 0.0001
NEW_WEIGHT_ROT_SMOOTHNESS = 0.00001

USE_COMMON_INTRINSICS = False

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


In [3]:
GAUSSIAN_SIGMA = 21.0
SIZE = int(GAUSSIAN_SIGMA * 3)

kernel = create_gaussian_kernel(SIZE, GAUSSIAN_SIGMA).to(device)

SIZE = 5
kernel = torch.ones(SIZE, SIZE).to(device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# Load the model

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [5]:
images_array = []
masks_array = []

for i in range(44):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}{}".format(i,IMG_FILE_EXTENSION)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}{}".format(i,MASK_FILE_EXTENSION)))
images = load_images(images_array, size=512, verbose=True)
_,_,H,W = images[0]["img"].shape
masks = load_masks(masks_array, H, W, device)

>> Loading a list of 44 images
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/0.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/1.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/2.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/3.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/4.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/5.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/6.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/7.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_i

In [6]:
pairs = make_pairs(images, scene_graph='swin-1', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, kernel, batch_size=batch_size)
del pairs

>> Inference with model on 88 image pairs


100%|██████████| 88/88 [00:36<00:00,  2.41it/s]


In [7]:
#init_scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer)
#loss = init_scene.compute_global_alignment(init="mst", niter=niter, schedule='cosine', lr=lr)

#scene = init_scene

scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer, 
                        weight_focal = INIT_WEIGHT_FOCAL,
                        weight_z = INIT_WEIGHT_Z ,
                        weight_rot = INIT_WEIGHT_ROT  ,
                        weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS,
                        weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS)
#scene.im_poses = calculate_new_params(init_scene.im_poses,device)
#scene.im_focals = init_scene.im_focals
loss = scene.compute_global_alignment(init="mst", niter=niter*10, schedule=schedule, lr=lr)

#averge_focal = scene.get_focals().sum().item()/len(images_array)
#fixed_focal = [averge_focal for _ in range(len(images_array))]
#mask = [True for _ in range(len(images_array))]
#scene.preset_focal(fixed_focal, mask)
#loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#scene.weight_focal = INIT_WEIGHT_FOCAL
#scene.weight_z = INIT_WEIGHT_Z 
#scene.weight_rot = INIT_WEIGHT_ROT
#scene.weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS 
#scene.weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS
#loss = global_alignment_loop(scene, niter=niter, schedule=schedule, lr=lr)


 init edge (27*,26*) score=1.360966682434082
 init edge (27,28*) score=1.3121287822723389
 init edge (26,25*) score=1.2196820974349976
 init edge (28,29*) score=1.203314185142517
 init edge (25,24*) score=1.190906286239624
 init edge (30*,29) score=1.1762253046035767
 init edge (30,31*) score=1.1614958047866821
 init edge (24,23*) score=1.155872106552124
 init edge (32*,31) score=1.1429954767227173
 init edge (23,22*) score=1.1675777435302734
 init edge (33*,32) score=1.1538861989974976
 init edge (34*,33) score=1.1501089334487915
 init edge (21*,22) score=1.1749271154403687
 init edge (21,20*) score=1.1550123691558838
 init edge (35*,34) score=1.154326319694519
 init edge (20,19*) score=1.1406059265136719
 init edge (18*,19) score=1.1363943815231323
 init edge (17*,18) score=1.188900351524353
 init edge (36*,35) score=1.1635130643844604
 init edge (17,16*) score=1.1471844911575317
 init edge (15*,16) score=1.1701997518539429
 init edge (36,37*) score=1.1692731380462646
 init edge (15,

100%|██████████| 3000/3000 [21:31<00:00,  2.32it/s, lr=1.00274e-06 loss=5.78471e-05]


In [8]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()
intrinsics = scene.get_intrinsics()

In [9]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

In [10]:
#Create transform file

img = Image.open(images_array[0])
width, height = img.size
RESCALE_FACTOR  = width/512

transforms = {}
transforms["camera_model"] = "OPENCV"
if USE_COMMON_INTRINSICS:
    intrinsic_mean = intrinsics.mean(dim=0)
    transforms["fl_x"] = intrinsic_mean[0,0].item() * RESCALE_FACTOR
    transforms["fl_y"] = intrinsic_mean[1,1].item() * RESCALE_FACTOR
    transforms["w"] = width 
    transforms["h"] = height 
    transforms["cx"] = intrinsic_mean[0,2].item() * RESCALE_FACTOR
    transforms["cy"] = intrinsic_mean[1,2].item() * RESCALE_FACTOR

transforms["frames"] = []

OPENGL = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])

for i in range(len(poses)):
    if not((confidence_masks[i]==0).all()):
        frame = {}
        frame["file_path"] = "/".join(images_array[i].split("/")[-2:])
        frame["transform_matrix"] = np.linalg.inv(poses[i].detach().cpu().numpy()).tolist()
        #frame["transform_matrix"] = np.dot(poses[i].detach().cpu().numpy(),OPENGL).tolist()
        frame["mask_path"] = "/".join(masks_array[i].split("/")[-2:])
        transforms["frames"].append(frame)
        
        if not USE_COMMON_INTRINSICS:
            frame["fl_x"] = intrinsics[i,0,0].item() * RESCALE_FACTOR
            frame["fl_y"] = intrinsics[i,1,1].item() * RESCALE_FACTOR
            frame["cx"] = intrinsics[i,0,2].item() * RESCALE_FACTOR
            frame["cy"] = intrinsics[i,1,2].item() * RESCALE_FACTOR
            img = Image.open(images_array[i])
            width, height = img.size
            transforms["w"] = width 
            transforms["h"] = height 

#Save transform file
with open("{}/transforms.json".format(DATA_PATH), 'w') as f:
    json.dump(transforms, f, indent=4)

In [11]:
import roma

all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize().to_homogeneous()

OPENGL = torch.tensor([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1]], dtype=torch.float32).to(device)

tf = torch.matmul(tf, OPENGL)

tf = roma.RigidUnitQuat(Q, T).normalize()
print(tf.linear[:,0]/tf.linear[:,3])

tensor([ 0.0992, -0.0907, -0.0914, -0.0017,  0.0217,  0.0216,  0.0330,  0.0329,
         0.0394,  0.0446,  0.0505,  0.0554,  0.0597,  0.0625,  0.0643,  0.0642,
         0.0660,  0.0685,  0.0704,  0.0728,  0.0761,  0.0793,  0.0813,  0.0832,
         0.0848,  0.0860,  0.0872,  0.0881,  0.0885,  0.0888,  0.0891,  0.0894,
         0.0896,  0.0899,  0.0902,  0.0906,  0.0911,  0.0917,  0.0924,  0.0933,
         0.0933,  0.0950,  0.0968,  0.0992], device='cuda:0',
       grad_fn=<DivBackward0>)
